In [1]:
from fpl import Loader, Player, Team, ExpectedPointsCalculator, Optimizer
from typing import List
import heapq

### Dowloading the team

In [2]:
picks = Loader.get_my_team_from_api(
    "aledvaghela@gmail.com", "NCP@jxy7pau-bvm3cuf", 3247546
)["picks"]
gkps, defs, mids, fwds = set(), set(), set(), set()
for pick in picks:
    d = {
        "element": pick["element"],
        "name": Loader.get_player_basic_info(pick["element"])["web_name"],
        "position": Loader.get_player_basic_info(pick["element"])["element_type"],
        "club": Loader.get_player_basic_info(pick["element"])["team"],
        "cost": pick["selling_price"],
    }
    player = Player(**d)
    if player.position == 1:
        gkps.add(player)
    if player.position == 2:
        defs.add(player)
    if player.position == 3:
        mids.add(player)
    if player.position == 4:
        fwds.add(player)

money_in_bank = Loader.get_my_team_from_api(
    "aledvaghela@gmail.com", "NCP@jxy7pau-bvm3cuf", 3247546
)["transfers"]["bank"]
free_transfers = Loader.get_my_team_from_api(
    "aledvaghela@gmail.com", "NCP@jxy7pau-bvm3cuf", 3247546
)["transfers"]["limit"]
free_transfers = 0 if free_transfers is None else free_transfers
team = Team(
    money_in_bank,
    free_transfers,
    frozenset(gkps),
    frozenset(defs),
    frozenset(mids),
    frozenset(fwds),
)
print(team)

---------------------------------------------------------------------------
{GKPS}
Player(element=201, name='Henderson', position=1, club=7, cost=44)
Player(element=521, name='Fabianski', position=1, club=19, cost=41)
---------------------------------------------------------------------------
{DEFS}
Player(element=18, name='Saliba', position=2, club=1, cost=61)
Player(element=70, name='Kerkez', position=2, club=3, cost=47)
Player(element=311, name='Alexander-Arnold', position=2, club=12, cost=72)
Player(element=399, name='Hall', position=2, club=15, cost=47)
Player(element=475, name='Taylor', position=2, club=17, cost=39)
---------------------------------------------------------------------------
{MIDS}
Player(element=182, name='Palmer', position=3, club=6, cost=109)
Player(element=247, name='Iwobi', position=3, club=9, cost=57)
Player(element=328, name='M.Salah', position=3, club=12, cost=132)
Player(element=366, name='B.Fernandes', position=3, club=14, cost=84)
Player(element=585, na

### Find a player

In [5]:
Loader.find_matching_players("rpe334")

ValueError: No matching players found.

### Candidate list

In [6]:
candidates_gkps = [413]
candidates_defs = [231, 350]
candidates_mids = [199, 503]
candidates_fwds = [58, 351]

### How to calculate points

In [7]:
class ComplexExpectedPointsCalculator(ExpectedPointsCalculator):
    minutes_multiplier = {
        # current team
        201: 90 / 90,  # Henderson
        521: 90 / 90,  # Fabianski
        18: 90 / 90,  # saliba
        70: 90 / 90,  # kerkez
        311: 90 / 90,  # arnold
        399: 90 / 90,  # hall
        475: 0,  # taylor
        182: 90 / 90,  # Palmer
        247: 90 / 90,  # Iwobi
        328: 90 / 90,  # M.Salah
        366: 90 / 90,  # B.Fernandes
        585: 90 / 90,  # I.Sarr
        129: 90 / 90,  # João Pedro
        401: 90 / 90,  # Isak
        447: 90 / 90,  # Wood
        # candidates
        413: 90 / 90,  # pope
        231: 90 / 90,  # mykolenko
        350: 90 / 90,  # gvardiol
        199: 90 / 90,  # eze
        503: 90 / 90,  # son
        58: 60 / 90,  # watkins
        351: 90 / 90,  # haaland
    }
    points_per_game_override = {
        503: 10.0,  # son
    }

    def get_expected_points(player_id: int, gameweek: int) -> float:
        points_per_game = 0
        element_type = Loader.get_player_basic_info(player_id)["element_type"]
        if player_id in ComplexExpectedPointsCalculator.points_per_game_override:
            points_per_game = ComplexExpectedPointsCalculator.points_per_game_override[
                player_id
            ]
        else:
            new_info = Loader.get_player_basic_info(player_id)
            points_per_90_this_season = float(new_info["points_per_game"])
            xP = 2.0
            xP += 4 * new_info["expected_assists_per_90"]
            if element_type == 1:
                xP += max(0, 4 * (1 - new_info["expected_goals_conceded_per_90"]))
                xP += 0.33 * new_info["saves_per_90"]
                xP += 6 * new_info["expected_goals_per_90"]
            elif element_type == 2:
                xP += max(0, 4 * (1 - new_info["expected_goals_conceded_per_90"]))
                xP += 6 * new_info["expected_goals_per_90"]
            elif element_type == 3:
                xP += max(0, 1 * (1 - new_info["expected_goals_conceded_per_90"]))
                xP += 5 * new_info["expected_goals_per_90"]
            elif element_type == 4:
                xP += 4 * new_info["expected_goals_per_90"]

            points_per_game = 0.25 * xP + 0.75 * points_per_90_this_season

        result = 0
        for fixture in Loader.get_player_future_info_for_gameweek(player_id, gameweek):
            our_team, opponent = -1, -1
            if fixture["is_home"]:
                our_team = fixture["team_h"]
                opponent = fixture["team_a"]
            else:
                our_team = fixture["team_a"]
                opponent = fixture["team_h"]

            difficulty_alpha = 0
            if fixture["difficulty"] == 2:
                difficulty_alpha = 1.5
            elif fixture["difficulty"] == 3:
                difficulty_alpha = 0.75
            elif fixture["difficulty"] == 4:
                difficulty_alpha = -0.75
            elif fixture["difficulty"] == 5:
                difficulty_alpha = -1.5
            else:
                raise ValueError("Fixture difficulty should be be 2,3,4,5")

            result += points_per_game + difficulty_alpha

        # for goalies assume they start and play 90
        if Loader.get_player_basic_info(player_id)["element_type"] == 1:
            return result

        if player_id not in ComplexExpectedPointsCalculator.minutes_multiplier:
            web_name = Loader.get_player_basic_info(player_id)["web_name"]
            raise KeyError(f"You need to get the minutes for {player_id} {web_name}")

        mm = ComplexExpectedPointsCalculator.minutes_multiplier[player_id]

        return result * mm

In [8]:
def get_discounted_reward(
    player_id: int, number_gameweeks: int, gamma: float = 1
) -> float:
    """Gets the discounted reward of a candidate over a certain number of gameweeks
    :param player_id: the player which you want the reward for
    :param number_gameweeks: how many gameweeks do you want to consider
    :param gamma: discount factor

    :return: discounted reward
    """
    discounted_reward = 0
    discount_factor = 1
    for gw in range(23, 23 + number_gameweeks):
        discounted_reward += (
            discount_factor
            * ComplexExpectedPointsCalculator.get_expected_points(player_id, gw)
        )
        discount_factor *= gamma

    return discounted_reward


#     return discounted_reward / Loader.get_player_basic_info(player_id)["now_cost"]


def print_sorted_candidates(
    candidates: List[int], number_gameweeks: int, gamma: float = 1
) -> None:
    """Print out the list of players and rewards they get
    :param candidates: list of player ids
    :param number_gameweeks: how many gameweeks do you want to consider
    :param gamma: discount factor
    """
    sorted_candidates = []
    for i in candidates:
        r = get_discounted_reward(i, number_gameweeks, gamma)
        sorted_candidates.append((i, r))

    sorted_candidates = reversed(sorted(sorted_candidates, key=lambda x: x[1]))
    for i, r in sorted_candidates:
        web_name = Loader.get_player_basic_info(i)["web_name"]
        print(f"id: {i}, name: {web_name}, reward: {r}")

    return

In [9]:
print_sorted_candidates(
    candidates_gkps + candidates_defs + candidates_mids + candidates_fwds, 1, 0.80
)

id: 503, name: Son, reward: 11.5
id: 199, name: Eze, reward: 5.422499999999999
id: 351, name: Haaland, reward: 5.12
id: 413, name: Pope, reward: 4.697000000000001
id: 58, name: Watkins, reward: 4.32
id: 231, name: Mykolenko, reward: 3.6450000000000005
id: 350, name: Gvardiol, reward: 2.985


### Optimizing the team

In [10]:
candidates = []
for pid in candidates_gkps + candidates_defs + candidates_mids + candidates_fwds:
    d = {
        "element": pid,
        "name": Loader.get_player_basic_info(pid)["web_name"],
        "position": Loader.get_player_basic_info(pid)["element_type"],
        "club": Loader.get_player_basic_info(pid)["team"],
        "cost": Loader.get_player_basic_info(pid)["now_cost"],
    }
    player = Player(**d)
    if player in gkps | defs | mids | fwds:
        print(f"Already have: {player}")
        continue
    else:
        candidates.append(player)

In [11]:
top_three = Optimizer.optimize_team(
    team,
    candidates,
    ComplexExpectedPointsCalculator,
    gameweek=23,
    horizon=3,
    max_transfers=2,
    gamma=0.8,
    wildcard=False,
)
third_best_score, third_best_team = heapq.heappop(top_three)
second_best_score, second_best_team = heapq.heappop(top_three)
first_best_score, first_best_team = heapq.heappop(top_three)

In [12]:
print("Score: {}".format(first_best_score))
print(first_best_team)

Score: 221.70520100000005
---------------------------------------------------------------------------
{GKPS}
Player(element=201, name='Henderson', position=1, club=7, cost=44)
Player(element=521, name='Fabianski', position=1, club=19, cost=41)
---------------------------------------------------------------------------
{DEFS}
Player(element=70, name='Kerkez', position=2, club=3, cost=47)
Player(element=231, name='Mykolenko', position=2, club=8, cost=43)
Player(element=311, name='Alexander-Arnold', position=2, club=12, cost=72)
Player(element=399, name='Hall', position=2, club=15, cost=47)
Player(element=475, name='Taylor', position=2, club=17, cost=39)
---------------------------------------------------------------------------
{MIDS}
Player(element=182, name='Palmer', position=3, club=6, cost=109)
Player(element=247, name='Iwobi', position=3, club=9, cost=57)
Player(element=328, name='M.Salah', position=3, club=12, cost=132)
Player(element=503, name='Son', position=3, club=18, cost=98)


In [13]:
print("Score: {}".format(second_best_score))
print(second_best_team)

Score: 218.028701
---------------------------------------------------------------------------
{GKPS}
Player(element=201, name='Henderson', position=1, club=7, cost=44)
Player(element=521, name='Fabianski', position=1, club=19, cost=41)
---------------------------------------------------------------------------
{DEFS}
Player(element=18, name='Saliba', position=2, club=1, cost=61)
Player(element=70, name='Kerkez', position=2, club=3, cost=47)
Player(element=231, name='Mykolenko', position=2, club=8, cost=43)
Player(element=311, name='Alexander-Arnold', position=2, club=12, cost=72)
Player(element=399, name='Hall', position=2, club=15, cost=47)
---------------------------------------------------------------------------
{MIDS}
Player(element=247, name='Iwobi', position=3, club=9, cost=57)
Player(element=328, name='M.Salah', position=3, club=12, cost=132)
Player(element=366, name='B.Fernandes', position=3, club=14, cost=84)
Player(element=503, name='Son', position=3, club=18, cost=98)
Playe

In [14]:
print("Score: {}".format(third_best_score))
print(third_best_team)

Score: 217.18710100000004
---------------------------------------------------------------------------
{GKPS}
Player(element=201, name='Henderson', position=1, club=7, cost=44)
Player(element=521, name='Fabianski', position=1, club=19, cost=41)
---------------------------------------------------------------------------
{DEFS}
Player(element=18, name='Saliba', position=2, club=1, cost=61)
Player(element=231, name='Mykolenko', position=2, club=8, cost=43)
Player(element=311, name='Alexander-Arnold', position=2, club=12, cost=72)
Player(element=399, name='Hall', position=2, club=15, cost=47)
Player(element=475, name='Taylor', position=2, club=17, cost=39)
---------------------------------------------------------------------------
{MIDS}
Player(element=247, name='Iwobi', position=3, club=9, cost=57)
Player(element=328, name='M.Salah', position=3, club=12, cost=132)
Player(element=366, name='B.Fernandes', position=3, club=14, cost=84)
Player(element=503, name='Son', position=3, club=18, cost

In [15]:
Optimizer.optimal_formation(first_best_team, ComplexExpectedPointsCalculator, 23)

{'gkps': frozenset({Player(element=201, name='Henderson', position=1, club=7, cost=44)}),
 'defs': frozenset({Player(element=70, name='Kerkez', position=2, club=3, cost=47),
            Player(element=311, name='Alexander-Arnold', position=2, club=12, cost=72),
            Player(element=399, name='Hall', position=2, club=15, cost=47)}),
 'mids': frozenset({Player(element=182, name='Palmer', position=3, club=6, cost=109),
            Player(element=247, name='Iwobi', position=3, club=9, cost=57),
            Player(element=328, name='M.Salah', position=3, club=12, cost=132),
            Player(element=503, name='Son', position=3, club=18, cost=98)}),
 'fwds': frozenset({Player(element=129, name='João Pedro', position=4, club=5, cost=56),
            Player(element=401, name='Isak', position=4, club=15, cost=92),
            Player(element=447, name='Wood', position=4, club=16, cost=70)}),
 'captain': Player(element=503, name='Son', position=3, club=18, cost=98),
 'total_exp_points': 86

In [16]:
Optimizer.optimal_formation(second_best_team, ComplexExpectedPointsCalculator, 23)

{'gkps': frozenset({Player(element=201, name='Henderson', position=1, club=7, cost=44)}),
 'defs': frozenset({Player(element=18, name='Saliba', position=2, club=1, cost=61),
            Player(element=311, name='Alexander-Arnold', position=2, club=12, cost=72),
            Player(element=399, name='Hall', position=2, club=15, cost=47)}),
 'mids': frozenset({Player(element=247, name='Iwobi', position=3, club=9, cost=57),
            Player(element=328, name='M.Salah', position=3, club=12, cost=132),
            Player(element=366, name='B.Fernandes', position=3, club=14, cost=84),
            Player(element=503, name='Son', position=3, club=18, cost=98)}),
 'fwds': frozenset({Player(element=129, name='João Pedro', position=4, club=5, cost=56),
            Player(element=401, name='Isak', position=4, club=15, cost=92),
            Player(element=447, name='Wood', position=4, club=16, cost=70)}),
 'captain': Player(element=503, name='Son', position=3, club=18, cost=98),
 'total_exp_points

In [17]:
Optimizer.optimal_formation(team, ComplexExpectedPointsCalculator, 23)

{'gkps': frozenset({Player(element=201, name='Henderson', position=1, club=7, cost=44)}),
 'defs': frozenset({Player(element=18, name='Saliba', position=2, club=1, cost=61),
            Player(element=311, name='Alexander-Arnold', position=2, club=12, cost=72),
            Player(element=399, name='Hall', position=2, club=15, cost=47)}),
 'mids': frozenset({Player(element=182, name='Palmer', position=3, club=6, cost=109),
            Player(element=247, name='Iwobi', position=3, club=9, cost=57),
            Player(element=328, name='M.Salah', position=3, club=12, cost=132),
            Player(element=366, name='B.Fernandes', position=3, club=14, cost=84)}),
 'fwds': frozenset({Player(element=129, name='João Pedro', position=4, club=5, cost=56),
            Player(element=401, name='Isak', position=4, club=15, cost=92),
            Player(element=447, name='Wood', position=4, club=16, cost=70)}),
 'captain': Player(element=328, name='M.Salah', position=3, club=12, cost=132),
 'total_ex